In [22]:
import bvhio

root_motionet = bvhio.readAsHierarchy('motionet_output.bvh')
print("motionet frames:", root_motionet.getKeyframeRange()[1] + 1)
root_motionet.loadPose(0)
hips_pos = root_motionet.filter('Hips')[0].PositionWorld
# print(hips_pos)
print('\nmotionet Position and Y-direction of each joint in world space ')
for joint, index, depth in root_motionet.layout():
    rel_pos = joint.PositionWorld - hips_pos
    # print(f'{joint.PositionWorld} {joint.Name}')
    print(f'{rel_pos} {joint.Name}')

root_tdpt = bvhio.readAsHierarchy('tdpt_output.bvh')
print("tdpt frames:", root_tdpt.getKeyframeRange()[1] + 1)
root_tdpt.loadPose(0)
hips_pos = root_tdpt.filter('Hips')[0].PositionWorld
# print(hips_pos)
print('\ntdpt Position and Y-direction of each joint in world space ')
for joint, index, depth in root_tdpt.layout():
    rel_pos = joint.PositionWorld - hips_pos
    # print(f'{joint.PositionWorld} {joint.Name}')
    print(f'{rel_pos} {joint.Name}')

motionet frames: 100

motionet Position and Y-direction of each joint in world space 
vec3(            0,            0,            0 ) Hips
vec3(     -2.90102,            0,            0 ) RightUpLeg
vec3(     -2.90102,     -9.11485,            0 ) RightLeg
vec3(     -2.90102,     -18.0173,            0 ) RightFoot
vec3(      2.90102,            0,            0 ) LeftUpLeg
vec3(      2.90102,     -9.11485,            0 ) LeftLeg
vec3(      2.90102,     -18.0173,            0 ) LeftFoot
vec3(            0,      5.15659,            0 ) Spine
vec3(            0,      10.2532,            0 ) Spine1
vec3(            0,      12.6667,            0 ) Neck
vec3(            0,      14.9915,            0 ) Head
vec3(      3.32228,      10.2532,            0 ) LeftArm
vec3(      9.12815,      10.2532,            0 ) LeftForeArm
vec3(      14.0901,      10.2532,            0 ) LeftHand
vec3(     -3.32228,      10.2532,            0 ) RightArm
vec3(     -9.12815,      10.2532,            0 ) RightFo

In [ ]:
def extract_keypoints(root):
    """Extract world-space keypoints for all frames"""
    num_frames = root.getKeyframeRange()[1] + 1
    keypoints = []

    for frame in range(num_frames):
        root.loadPose(frame)
        keypoints.append([joint.Name, joint.PositionWorld]
                         for joint, index, depth in root.layout())

    return np.array(keypoints)

In [24]:
import bvhio
import numpy as np

# --- MOTIONET ---
root_motionet = bvhio.readAsHierarchy('motionet_output.bvh')
root_motionet.loadPose(0)
hips_pos = root_motionet.filter('Hips')[0].PositionWorld

motionet_rel = []
print('\nmotionet Position (mm, hips-relative):')
for joint, index, depth in root_motionet.layout():
    rel_pos = joint.PositionWorld - hips_pos
    rel_pos_mm = np.array([rel_pos.x, rel_pos.y, rel_pos.z]) * 10.0  # cm to mm
    motionet_rel.append(rel_pos_mm)
    print(f'{rel_pos_mm} {joint.Name}')

# --- TDPT ---
root_tdpt = bvhio.readAsHierarchy('tdpt_output.bvh')
root_tdpt.loadPose(0)
hips_pos = root_tdpt.filter('Hips')[0].PositionWorld

tdpt_rel = []
print('\ntdpt Position (mm, hips-relative):')
for joint, index, depth in root_tdpt.layout():
    rel_pos = joint.PositionWorld - hips_pos
    rel_pos_mm = np.array(
        [rel_pos.x, rel_pos.y, rel_pos.z]) * 1000.0  # m to mm
    tdpt_rel.append(rel_pos_mm)
    print(f'{rel_pos_mm} {joint.Name}')

# --- VERIFY MATCHING ---
motionet_rel = np.array(motionet_rel)
tdpt_rel = np.array(tdpt_rel)

# If the joint order is the same, you can directly compare:
print("\nEuclidean distance between corresponding joints (in mm):")
for i in range(min(len(motionet_rel), len(tdpt_rel))):
    dist = np.linalg.norm(motionet_rel[i] - tdpt_rel[i])
    print(f"{root_motionet.layout()[i][0].Name}: {dist:.2f} mm")


motionet Position (mm, hips-relative):
[0. 0. 0.] Hips
[-29.01024103   0.           0.        ] RightUpLeg
[-29.01024103 -91.14854813   0.        ] RightLeg
[ -29.01024103 -180.17313004    0.        ] RightFoot
[29.01024103  0.          0.        ] LeftUpLeg
[ 29.01024103 -91.14854813   0.        ] LeftLeg
[  29.01024103 -180.17313004    0.        ] LeftFoot
[ 0.        51.5658617  0.       ] Spine
[  0.         102.53236771   0.        ] Spine1
[  0.         126.66733742   0.        ] Neck
[  0.        149.9154377   0.       ] Head
[ 33.22279692 102.53236771   0.        ] LeftArm
[ 91.2815094  102.53237724   0.        ] LeftForeArm
[140.90118408 102.53238678   0.        ] LeftHand
[-33.22279692 102.53236771   0.        ] RightArm
[-91.2815094  102.53237724   0.        ] RightForeArm
[-140.90118408  102.53238678    0.        ] RightHand

tdpt Position (mm, hips-relative):
[0. 0. 0.] Hips
[ 0.         58.57741833 14.13106918] Spine
[  0.         185.22024155  17.36694574] Chest
[  0.  